### ทส525: วิทยาการข้อมูล

<b>Supanat Jintawatsakoon</b><br>

In [1]:
#!pip install -U nltk

In [2]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
import collections

In [3]:
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import csv
from sklearn.svm import LinearSVC, SVC
import nltk.classify.util, nltk.metrics

In [4]:
# Set Current Directory
currentPath = os.getcwd()
subFolder = "/trainingandtestdata/"
os.chdir(currentPath+subFolder)

In [5]:
# Dataset : Go to http://help.sentiment140.com/for-students/
train_f = 'training.1600000.processed.noemoticon.csv'
test_f = 'testdata.manual.2009.06.14.csv'
cols = ['sentiment','id','date','query_string','user','text']
train_df = pd.read_csv(train_f, header=None, names=cols, encoding='ISO-8859-1')
test_df = pd.read_csv(test_f, header=None, names=cols, encoding='ISO-8859-1')

In [6]:
train_df.shape

(1600000, 6)

In [7]:
test_df.shape

(498, 6)

In [8]:
train_df.columns.tolist()

['sentiment', 'id', 'date', 'query_string', 'user', 'text']

In [9]:
test_df.columns.tolist()

['sentiment', 'id', 'date', 'query_string', 'user', 'text']

In [10]:
train_df.dtypes

sentiment        int64
id               int64
date            object
query_string    object
user            object
text            object
dtype: object

In [11]:
train_df.head(5)

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
train_df.tail(5)

,sentiment,id,date,query_string,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [13]:
train_df['sentiment'].value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [14]:
selected_columns = ['sentiment', 'text']
trainset = train_df[selected_columns].copy()

In [15]:
testset = test_df[selected_columns].copy()

In [16]:
train_df.columns.tolist()


['sentiment', 'id', 'date', 'query_string', 'user', 'text']

In [17]:
cols_to_drop = ['id', 'date', 'query_string', 'user']
train_df.drop(cols_to_drop, axis=1, inplace=True)
train_df.columns.tolist()

['sentiment', 'text']

In [18]:
trainset.shape

(1600000, 2)

In [19]:
trainset.columns.tolist()

['sentiment', 'text']

In [20]:
trainset.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [21]:
text = trainset.loc[343, 'text']
text

'@TheLeagueSF Not Fun &amp; Furious? The new mantra for the Bay 2 Breakers? It was getting 2 rambunctious;the city overreacted &amp; clamped down '

Define Function

In [22]:
def convertHTMLEncodedToNormalCharacter(text):
    soup = BeautifulSoup(text, 'lxml')
    return soup.get_text()

In [23]:
convertHTMLEncodedToNormalCharacter(text)

'@TheLeagueSF Not Fun & Furious? The new mantra for the Bay 2 Breakers? It was getting 2 rambunctious;the city overreacted & clamped down '

In [24]:
#detect http://, https://, www. and www2.
pattern = r'(https?://[A-Za-z0-9./]+|www[\d]?\.[^ ]+)'
regex = re.compile(pattern, flags=0)
for row, text in zip(trainset['text'].index, trainset.loc[0:100, 'text']):
    matchedList = regex.findall(text) 
    for obj in matchedList:
        print("Row is {0} and text is {1}".format(row, obj))

Row is 0 and text is http://twitpic.com/2y1zl
Row is 37 and text is http://is.gd/fNge
Row is 50 and text is http://tinyurl.com/dcuc33
Row is 50 and text is www.diigo.com/~tautao
Row is 74 and text is http://twitpic.com/2y2es
Row is 95 and text is http://apps.facebook.com/dogbook/profile/view/5248435
Row is 100 and text is http://tr.im/imji


In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/supanat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
positiveTweets = trainset[trainset['sentiment']==4]
negativeTweets = trainset[trainset['sentiment']==0]

In [27]:
positiveSample = positiveTweets.sample(
     n=500, #number of items from axis to return. 1 if n=None frac=None, #Fraction of axis to return
     replace=False, #sampling without replacement
     weights=None, #equal probablity weighting
     random_state=181, #seed
     axis=None)#0 for series and data frames (default), 1 for panels

negativeSample = negativeTweets.sample(
     n=500, #number of items from axis to return. 1 if n=None frac=None, #Fraction of axis to return
     replace=False, #sampling without replacement
     weights=None, #equal probablity weighting
     random_state=181, #seed
     axis=None)#0 for series and data frames (default), 1 for panels

In [28]:
positiveSample.shape

(500, 2)

In [29]:
negativeSample.shape

(500, 2)

In [30]:
trainSample = positiveSample.append(negativeSample)
trainSample.shape

(1000, 2)

In [31]:
def bagOfWords(tweets): 
    wordsList = []
    for (words, sentiment) in tweets:
        wordsList.extend(words)
    return wordsList


In [32]:
def wordFeatures(wordList):
    wordList = nltk.FreqDist(wordList)
    wordFeatures = wordList.keys()
    return wordFeatures

In [33]:
def getFeatures(doc):
    docWords = set(doc)
    features = {}
    for word in wordFeatures:
        features['contains(%s)' % word] = (word in docWords)
    return features

Construct a train corpus

In [34]:
trainCorpusOfTweets = []
for (text, sentiment) in (zip(trainSample.text, trainSample.sentiment)):
    wordsFiltered = [e.lower() for e in nltk.word_tokenize(text) if len(e) >= 3] 
    trainCorpusOfTweets.append((wordsFiltered, sentiment))

In [35]:
for vsm in trainCorpusOfTweets: 
    print(" {0}\n".format(vsm))

 (['robintile', 'yeah', 'you', 'got', 'pic'], 4)

 (['kelly960', 'you', "'re", 'welcome', 'you', "'ve", 'done', 'enough', 'for'], 4)

 (['comptonassneil', 'thank', 'you'], 4)

 (['imodel_', 'aww', 'yeah', 'was', 'that', 'way', 'the', 'boutique', '*tear*'], 4)

 (['senior', 'wow', "n't", 'feel', 'important', 'like', 'thought', 'would', 'kinda', 'sad'], 4)

 (['great', 'day', 'until', 'now'], 4)

 (['scottyksawgrass', 'alright', 'either', 'way', 'thn', 'cuz', 'sunday', 'therush', 'wuld', 'rly', 'like', 'some', 'cookies', 'famly', 'doesnt', 'eat', "'em", 'all'], 4)

 (['holy', 'balls', 'already', 'pissing', 'pants', 'from', 'the', 'mtv', 'movie', 'awards', 'andy', 'hero'], 4)

 (['djjunebug', 'nice', 'pic', 'aha', 'working', 'hard', '...'], 4)

 (['erickthegreat', 'really', 'was', 'yummy'], 4)

 (['cuppycake143', 'lol', 'sounds', 'like', 'when', 'was', 'chillin', 'with', 'jerson', 'got', 'love', 'them', 'old', 'school', 'jams'], 4)

 (['hermanatrix', 'the', 'ride', 'today', 'was', 'surpri

 (['sometimes', 'hate', 'having', 'friends', 'dramatic', 'and', 'annoying', 'and', 'best', 'friend', 'got', 'fight', 'was', 'refreshingly', 'alone', 'now', 'over'], 0)

 (['top', 'dropkick', 'murphys', 'were', 'good', 'though', 'getting', 'grub', 'heading', 'home', 'feeling', 'alright'], 0)

 (['tangg', 'holla', 'from', 'scotland', 'wots', 'the', 'tartan', 'army', 'just', 'returned', 'home', 'from', 'and'], 0)

 (['chrisbaccam', 'hate', 'you', 'wan', 'signed', 'one'], 0)

 (['hey', 'all', 'hope', 'you', 'all', 'have', 'wonderful', 'day', 'waiting', 'for', 'the', 'carpet', 'put', 'then', 'off', 'universal', 'still', 'sad', 'about', 'tampa', 'tonight'], 0)

 (['mdholla', 'morning', '...', 'enjoy', 'the', 'rainy', 'day'], 0)

 (['want', 'stolen', 'quot', 'monitor', 'back', 'testing', 'surface', 'apps', 'after', 'work', 'impossible', 'without', 'laptop', 'resolution', 'too', 'quot', 'low', 'quot'], 0)

 (['turns', 'out', 'today', 'not', 'that', 'bad', 'played', 'with', 'mrs_ricky', 'puppie

In [36]:
wordFeatures = wordFeatures(bagOfWords(trainCorpusOfTweets))

In [37]:
trainVSM = nltk.classify.apply_features(getFeatures, trainCorpusOfTweets)

In [38]:
classifier = nltk.NaiveBayesClassifier.train(trainVSM)

In [39]:
print(classifier.show_most_informative_features(25))

Most Informative Features
           contains(sad) = True                0 : 4      =      9.0 : 1.0
        contains(thanks) = True                4 : 0      =      9.0 : 1.0
          contains(hate) = True                0 : 4      =      7.8 : 1.0
         contains(about) = True                0 : 4      =      6.7 : 1.0
           contains(yay) = True                4 : 0      =      6.3 : 1.0
         contains(tired) = True                0 : 4      =      6.3 : 1.0
          contains(nice) = True                4 : 0      =      6.2 : 1.0
           contains('re) = True                4 : 0      =      5.8 : 1.0
           contains(bad) = True                0 : 4      =      5.8 : 1.0
     contains(followers) = True                4 : 0      =      5.7 : 1.0
         contains(thank) = True                4 : 0      =      5.4 : 1.0
          contains(sick) = True                0 : 4      =      5.0 : 1.0
          contains(does) = True                0 : 4      =      5.0 : 1.0

Construct a test corpus

In [40]:
testset = testset[(testset['sentiment']==0) | (testset['sentiment']==4)]
testCorpusOfTweets = []
for (text, sentiment) in (zip(testset.text, testset.sentiment)):
    wordsFiltered = [e.lower() for e in nltk.word_tokenize(text) if len(e) >= 3] 
    testCorpusOfTweets.append((wordsFiltered, sentiment))

In [41]:
testVSM = nltk.classify.apply_features(getFeatures, testCorpusOfTweets)

In [52]:
classifier = SklearnClassifier(RandomForestClassifier(random_state=0), sparse=False)
classifier.train(trainVSM)

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
 
for i, (text, sentiment) in enumerate(testVSM):
    refsets[sentiment].add(i)
    observed = classifier.classify(text)
    testsets[observed].add(i)
        
accuracy = nltk.classify.util.accuracy(classifier, testVSM)   
precision = (nltk.precision(refsets[0], testsets[0]) +  nltk.precision(refsets[4], testsets[4]))/2     
recall = (nltk.recall(refsets[0], testsets[0]) + nltk.recall(refsets[4], testsets[4]))/2

print('Accuracy :',accuracy)
print('Precision :',precision)
print('Recall :',recall)

Accuracy : 0.657381615599
Precision : 0.6596135571745327
Recall : 0.6564226733718259


In [54]:
classifier = SklearnClassifier(LinearSVC(), sparse=False)
classifier.train(trainVSM)


refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
 
for i, (text, sentiment) in enumerate(testVSM):
    refsets[sentiment].add(i)
    observed = classifier.classify(text)
    testsets[observed].add(i)
        
accuracy = nltk.classify.util.accuracy(classifier, testVSM)   
precision = (nltk.precision(refsets[0], testsets[0]) +  nltk.precision(refsets[4], testsets[4]))/2     
recall = (nltk.recall(refsets[0], testsets[0]) + nltk.recall(refsets[4], testsets[4]))/2

print('Accuracy :',accuracy)
print('Precision :',precision)
print('Recall :',recall)

Accuracy : 0.707520891365
Precision : 0.7081746675822461
Recall : 0.7070373129695164


----